In [1]:
import cv2
import numpy as np
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [2]:
image= cv2.imread("front_3.png")
#read the image, turn into grayscale and gradient threshold by keeping 0.3% of the highest gradient
image_grey = cv2.imread("front_3.png",cv2.IMREAD_GRAYSCALE)
image_grey = cv2.GaussianBlur(image_grey, (3, 3), 15)

gx= cv2.Scharr(np.float32(image_grey), cv2.CV_64F, 1, 0) #i need to understand these because idk     
gy= cv2.Scharr(np.float32(image_grey), cv2.CV_64F, 0, 1)

grad_mag = np.sqrt((gx ** 2) + (gy ** 2))

per= np.percentile(grad_mag, 99.7)


for i in range(grad_mag.shape[0]):
    for j in range(grad_mag.shape[1]):
        if(grad_mag[i, j]<per): #580
            grad_mag[i, j]=0 

    
ret, thresh1 = cv2.threshold(grad_mag, 240, 255, cv2.THRESH_BINARY)

In [3]:
thresh1=thresh1.astype(np.uint8)
cv2.imwrite('thresh1.png', thresh1)

True

In [4]:
#increase the thickness of the thresholded to aid in Canny edge detection
def increase_line_thickness(image, thickness):
    # Create a line-shaped structuring element
    structuring_element = cv2.getStructuringElement(cv2.MORPH_RECT, (1, thickness))
    
    # Perform dilation to increase line thickness
    dilated_image = cv2.dilate(image, structuring_element, iterations=1)
    
    return dilated_image

In [5]:
# Read the image
original_image = cv2.imread('thresh1.png', cv2.IMREAD_GRAYSCALE)

# Set the desired line thickness
thickness = 8

# Increase line thickness
thickened_image = increase_line_thickness(original_image, thickness)

# Perform Canny edge detection on the original image
canny_edges = cv2.Canny(original_image, 50, 300)

# Combine thickened image with Canny edges (this aids in helping Hough Line transform detect the horizontal lines)
combined_edges = cv2.bitwise_or(thickened_image, canny_edges)

In [43]:
image_grey = cv2.imread("front_3.png",cv2.IMREAD_GRAYSCALE)
image_grey = cv2.GaussianBlur(image_grey, (3, 3), 15)

gx= cv2.Scharr(np.float32(image_grey), cv2.CV_64F, 1, 0) #i need to understand these because idk     
gy= cv2.Scharr(np.float32(image_grey), cv2.CV_64F, 0, 1)

grad_mag = np.sqrt((gx ** 2) + (gy ** 2))

per= np.percentile(grad_mag, 99.6)


for i in range(grad_mag.shape[0]):
    for j in range(grad_mag.shape[1]):
        if(grad_mag[i, j]<per): #580
            grad_mag[i, j]=0 
ret, thresh1 = cv2.threshold(grad_mag, 240, 255, cv2.THRESH_BINARY)

thresh1=thresh1.astype(np.uint8)

In [44]:
cv2.imwrite('thresh1.png', combined_edges)

True

In [6]:
edges = cv2.Canny(thresh1, 100, 200)

In [7]:
# Apply Hough Line Transform
lines = cv2.HoughLinesP(edges, rho=5, theta=np.pi/180, threshold=1, minLineLength=10, maxLineGap=10)

In [8]:
# Filter and draw horizontal lines

for line in lines:
    x1, y1, x2, y2 = line[0]
    if y2>=750: 
        x1, y1, x2, y2= (0, 0, 0, 0)
    
    angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
    if np.abs(angle) < 40:  # Tolerance for horizontal lines
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

In [9]:
cv2.imshow('Horizontal Lines2', image)
cv2.waitKey(15)
cv2.destroyAllWindows()